In [5]:
import os 
os.getcwd()

'C:\\Users\\dell\\Documents\\iNeuron\\simple-dvc-demo\\notebooks'

In [9]:
import argparse
import os
from pprint import pprint

import joblib
import mlflow
from mlflow.tracking import MlflowClient

from src.get_data import read_params

os.chdir("\\Users\\dell\\Documents\\iNeuron\\simple-dvc-demo")


def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config["mlflow_config"]

    model_name = mlflow_config["registered_model_name"]

    remote_server_uri = mlflow_config["remote_server_uri"]
    mlflow_config.set_registry_uri(remote_server_uri)

    runs = mlflow.search_runs(experiment_ids=1)
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=model_name, version=current_version, stage="Production"
            )
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(
                name=model_name, version=current_version, stage="Staging"
            )
    loaded_model = mlflow.pyfunc.load_model(logged_model)
    model_path = config["webapp_model_dir"]

    joblib.dump(loaded_model, model_path)

FileNotFoundError: [Errno 2] No such file or directory: '../params.yaml'